In [2]:
pip install dash dash-bootstrap-components pandas plotly yfinance


In [1]:
pip install dash dash-bootstrap-components pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [3]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR

# Initialize Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# List of cryptocurrencies
crypto_assets = ["BTC-USD", "ETH-USD", "WBTC-USD", "STETH-USD"]

def fetch_data(ticker):
    """Fetch historical data for the selected cryptocurrency."""
    data = yf.download(ticker, period="2y", interval="1d")
    return data

def calculate_correlation():
    """Fetch data for all cryptos and compute correlation matrix."""
    df = pd.concat([fetch_data(ticker)["Close"] for ticker in crypto_assets], axis=1, keys=crypto_assets)
    return df.corr()

def predict_trend(data):
    """Simple Moving Average (SMA) Prediction."""
    data["SMA_30"] = data["Close"].rolling(window=30).mean()
    return data

def machine_learning_forecast(data):
    """Forecast future prices using Support Vector Regression (SVR)."""
    data = data.dropna()
    X = np.arange(len(data)).reshape(-1, 1)
    y = data["Close"].values

    scaler = MinMaxScaler()
    y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()

    X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, shuffle=False)
    model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
    model.fit(X_train, y_train)

    y_pred = scaler.inverse_transform(model.predict(X_test).reshape(-1, 1)).flatten()
    future_dates = data.index[-len(y_test):]

    return future_dates, y_test, y_pred

# Layout of the dashboard
app.layout = dbc.Container([
    html.H1("Cryptocurrency Dashboard", className="text-center mt-4"),

    dcc.Dropdown(
        id="crypto-dropdown",
        options=[{"label": asset, "value": asset} for asset in crypto_assets],
        value="BTC-USD",
        className="mb-4"
    ),

    dcc.Graph(id="price-chart"),
    dcc.Graph(id="correlation-heatmap"),
    dcc.Graph(id="forecast-chart"),

    html.H3("Descriptive Statistics"),
    dcc.Loading(dcc.Markdown(id="stats-output")),
], fluid=True)

# Callback to update the chart, heatmap, and stats
@app.callback(
    [Output("price-chart", "figure"), Output("correlation-heatmap", "figure"), Output("forecast-chart", "figure"), Output("stats-output", "children")],
    [Input("crypto-dropdown", "value")]
)
def update_dashboard(selected_crypto):
    data = fetch_data(selected_crypto)
    data = predict_trend(data)

    # Create price chart with trendline
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data["Close"], mode='lines', name='Close Price'))
    fig.add_trace(go.Scatter(x=data.index, y=data["SMA_30"], mode='lines', name='30-Day SMA', line=dict(dash='dash')))
    fig.update_layout(title=f"{selected_crypto} Closing Prices with 30-Day SMA")

    # Generate correlation heatmap
    corr_matrix = calculate_correlation()
    heatmap_fig = px.imshow(corr_matrix, text_auto=True, title="Cryptocurrency Correlation Heatmap")

    # Machine learning forecast
    future_dates, y_test, y_pred = machine_learning_forecast(data)
    forecast_fig = go.Figure()
    forecast_fig.add_trace(go.Scatter(x=future_dates, y=y_test, mode='lines', name='Actual Price'))
    forecast_fig.add_trace(go.Scatter(x=future_dates, y=y_pred, mode='lines', name='Predicted Price', line=dict(dash='dash')))
    forecast_fig.update_layout(title=f"{selected_crypto} Forecasted Prices")

    # Generate descriptive stats
    stats = data.describe().to_markdown()

    return fig, heatmap_fig, forecast_fig, f"""```
{stats}
```"""

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>